In [1]:
import pandas as pd
import seaborn as sns

# Cargar datos (ejemplo)
df = sns.load_dataset('titanic')
print("--- Antes de la limpieza ---")
print(df.isnull().sum()) 

# 1. DIAGNÓSTICO
print("--- Datos faltantes por columna ---")
print(df.isnull().sum())

# 2. ELIMINAR (Cirugía)
# La columna 'deck' (cubierta) tiene demasiados huecos. La matamos.
# axis=1 (columna), thresh=500 (borrar si tiene menos de 500 datos válidos)
df_limpio = df.dropna(axis=1, thresh=500)

# 3. RELLENAR NÚMEROS (Media)
# A 'age' le faltan datos, pero es importante. Rellenamos con el promedio.
promedio_edad = df['age'].mean()
df['age'].fillna(promedio_edad, inplace=True)

# 4. RELLENAR CATEGORÍAS (Moda)
# A 'embark_town' le faltan poquitos. Asumimos la ciudad más común.
ciudad_comun = df['embark_town'].value_counts().idxmax()
df['embark_town'].fillna(ciudad_comun, inplace=True)

print("\n--- Después de la limpieza ---")
print(df.isnull().sum()) # Debería salir casi todo en 0

--- Antes de la limpieza ---
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
--- Datos faltantes por columna ---
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

--- Después de la limpieza ---
survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      0
alive            0
alone            0
dtype: int64


C:\Users\alere\AppData\Local\Temp\ipykernel_21092\94053324.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(promedio_edad, inplace=True)
C:\Users\alere\AppData\Local\Temp\ipykernel_21092\94053324.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np

# --- 1. CARGA ---
df = sns.load_dataset('titanic')
print("--- 1. Datos Faltantes Iniciales ---")
print(df.isnull().sum())

# --- 2. LIMPIEZA (Sin Warnings) ---

# A) Eliminar columna 'deck' (porque faltan demasiados datos)
df_limpio = df.dropna(axis=1, thresh=500)

# B) Eliminar filas sin 'age' (en una copia aparte para probar)
df_sin_filas = df.dropna(subset=['age'], axis=0)

# --- 3. IMPUTACIÓN (Rellenar) ---
df_rellenado = df.copy()

# C) Rellenar Edad con Promedio (FORMA MODERNA)
promedio_edad = df['age'].mean()
# Así se hace ahora para evitar el warning:
df_rellenado['age'] = df_rellenado['age'].fillna(promedio_edad)

# D) Rellenar Ciudad con Moda (FORMA MODERNA)
ciudad_comun = df['embark_town'].value_counts().idxmax()
# Así se hace ahora:
df_rellenado['embark_town'] = df_rellenado['embark_town'].fillna(ciudad_comun)

# --- 4. VERIFICACIÓN FINAL ---
print("\n--- Resultado Final (Ceros = Limpio) ---")
# Solo mostramos las columnas que tocamos para no marearte
print(df_rellenado[['age', 'embark_town']].isnull().sum())

--- 1. Datos Faltantes Iniciales ---
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

--- Resultado Final (Ceros = Limpio) ---
age            0
embark_town    0
dtype: int64


In [4]:
import pandas as pd

# --- 1. CREACIÓN DE DATOS SUCIOS ---
data = {
    'ID': [101, 102, 103, 101, 104, 102],
    'Nombre': ['Ana', 'Beto', 'Carla', 'Ana', 'David', 'Beto'],
    'Email': ['ana@test.com', 'beto@test.com', 'carla@test.com', 'ana@test.com', 'david@test.com', 'beto@nuevo.com']
}

df = pd.DataFrame(data)

print("--- 1. DataFrame Original (Con Duplicados) ---")
print(df)
print("-" * 30)


# --- 2. DETECCIÓN (El Diagnóstico) ---
# duplicated() devuelve True/False.
# duplicated().sum() nos dice cuántos hay.
num_duplicados = df.duplicated().sum()
print(f"Filas duplicadas EXACTAS: {num_duplicados}")

# Truco Pro: Ver cuáles son las duplicadas
print("\n--- Mostrando las filas que son copias ---")
print(df[df.duplicated()]) 
# Nota: Verás que sale el ID 101 (Ana) porque es idéntica a la primera.
# Pero NO sale el ID 102 (Beto) porque su email cambió, así que para Pandas no son idénticos.


# --- 3. LIMPIEZA NIVEL 1: Duplicados Exactos ---
# drop_duplicates() borra solo si TODAS las columnas coinciden
df_limpio_exacto = df.drop_duplicates()

print(f"\n--- 2. Limpieza Exacta (Se fue la copia de Ana) ---")
print(df_limpio_exacto)


# --- 4. LIMPIEZA NIVEL 2: Duplicados por Columna (Subset) ---
# Problema: Beto (102) aparece dos veces con emails distintos. 
# Regla de negocio: "Un ID debe ser único".
# Usamos 'subset' para decirle: "Si el ID se repite, bórralo".

# keep='first' (por defecto): Se queda con el primero que encontró.
# keep='last': Se queda con el último (útil si quieres el dato más reciente).

df_sin_repes_id = df.drop_duplicates(subset=['ID'], keep='first')

print(f"\n--- 3. Limpieza por ID (Se fue el segundo Beto) ---")
print(df_sin_repes_id)

--- 1. DataFrame Original (Con Duplicados) ---
    ID Nombre           Email
0  101    Ana    ana@test.com
1  102   Beto   beto@test.com
2  103  Carla  carla@test.com
3  101    Ana    ana@test.com
4  104  David  david@test.com
5  102   Beto  beto@nuevo.com
------------------------------
Filas duplicadas EXACTAS: 1

--- Mostrando las filas que son copias ---
    ID Nombre         Email
3  101    Ana  ana@test.com

--- 2. Limpieza Exacta (Se fue la copia de Ana) ---
    ID Nombre           Email
0  101    Ana    ana@test.com
1  102   Beto   beto@test.com
2  103  Carla  carla@test.com
4  104  David  david@test.com
5  102   Beto  beto@nuevo.com

--- 3. Limpieza por ID (Se fue el segundo Beto) ---
    ID Nombre           Email
0  101    Ana    ana@test.com
1  102   Beto   beto@test.com
2  103  Carla  carla@test.com
4  104  David  david@test.com
